### 1. Описание набора данных

### 2. Импорт библиотек

In [3]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import urllib.request
import warnings

# from scipy.stats import yeojohnson
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.compose import ColumnTransformer
# from sklearn.metrics import mean_squared_error as mse
# from sklearn.metrics import PredictionErrorDisplay
# from sklearn.metrics import r2_score
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import ShuffleSplit
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.tree import DecisionTreeRegressor

In [4]:
warnings.filterwarnings('ignore')
logging.getLogger('matplotlib').setLevel(logging.ERROR)

### 3. Определение функций

### 4. Создание каталога и скачивание данных

In [5]:
if not os.path.exists('data'):
  os.makedirs('data')

if not os.path.exists('data/german_credit.csv'):
  urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/mbml_II_hw/refs/heads/main/lab3/data/german_credit.csv', 'data/german_credit.csv')

HTTPError: HTTP Error 404: Not Found

### 5. Загрузка набора данных из файла

In [ ]:
df = pd.read_csv('data/housing.csv')
df.head()